In [35]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from collections import OrderedDict

#remove warnings
import warnings
warnings.filterwarnings('ignore')

In [36]:
#define a coder and decoder for string to int and int to string
coder = OrderedDict()
decoder = OrderedDict()

def encode(s):
    if s not in coder:
        coder[s] = len(coder)
        decoder[len(decoder)] = s
    return coder[s]

def decode(i):
    return decoder[i]

#encode playerID, teamID
def encode_player_team(df):
    if('playerID' in df.columns):
        df['playerID'] = df['playerID'].apply(encode)
    if('tmID' in df.columns):
        df['tmID'] = df['tmID'].apply(encode)
    if('bioID' in df.columns):
        df['bioID'] = df['bioID'].apply(encode)
    if('college' in df.columns):
        df['college'] = df['college'].apply(encode)
    if 'confID' in df.columns:
        df['confID'] = df['confID'].apply(encode)
    return df


#get player teams into dataframe from player_teams.csv
player_teams = pd.read_csv('new_data/data.csv')

#get player teams into dataframe from player_teams.csv
coaches = pd.read_csv('../data/coaches.csv')

#teams match up results (only post is available)
series_post = pd.read_csv('../data/series_post.csv')

#team stats in playoff
teams_post = pd.read_csv('../data/teams_post.csv')

#player csv
players = pd.read_csv('../data/players.csv')

#awards csv
awards_players = pd.read_csv('../data/awards_players.csv')

#teams csv
teams = pd.read_csv('../data/teams.csv')

#comp csv
comp = pd.read_csv('new_data/comp.csv')

#remove all columns starting with "lgID" from all dataframes
for df in [player_teams, coaches, series_post, teams_post, players, awards_players, teams]:
    for col in df.columns:
        if col.startswith('lgID'):
            df.drop(col, axis=1, inplace=True)

#drop divID column from teams
teams.drop('divID', axis=1, inplace=True)

#drop all stint > 1 from player_teams
player_teams = player_teams[player_teams['stint'] == 0]

print(teams['confID'].unique())

['EA' 'WE']


In [37]:
def rolling_averages(group, cols, new_cols, window=2):
    group = group.sort_values('year')
    rolling_stats = group[cols].rolling(window=window, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [38]:
def compute_percentage(numerator, denominator):
    return round(numerator.divide(denominator).where(denominator != 0, 0.0),2)

In [39]:
# Assuming you have a column 'year' to sort by
player_teams = player_teams.sort_values(by=['playerID', 'year'])

# Regular Season Percentages
player_teams['ft%'] = compute_percentage(player_teams['ftMade'], player_teams['ftAttempted'])
player_teams['fg%'] = compute_percentage(player_teams['fgMade'], player_teams['fgAttempted'])
player_teams['three%'] = compute_percentage(player_teams['threeMade'], player_teams['threeAttempted'])
player_teams['gs%'] = compute_percentage(player_teams['GS'], player_teams['GP'])

# Playoffs Percentages
player_teams['Postft%'] = compute_percentage(player_teams['PostftMade'], player_teams['PostftAttempted'])
player_teams['Postfg%'] = compute_percentage(player_teams['PostfgMade'], player_teams['PostfgAttempted'])
player_teams['Postthree%'] = compute_percentage(player_teams['PostthreeMade'], player_teams['PostthreeAttempted'])
player_teams['Postgs%'] = compute_percentage(player_teams['PostGS'], player_teams['PostGP'])

#effective field goal percentage
player_teams['efg%'] = compute_percentage(player_teams['fgMade'] + 0.5 * player_teams['threeMade'], player_teams['fgAttempted']) * 2

#true shooting percentage
player_teams['ts%'] = compute_percentage(player_teams['points'], 2 * (player_teams['fgAttempted'] + 0.44 * player_teams['ftAttempted'])) * 2

#per game stats
#if pos contains G, then multiply by assists by 1.5, if contains C, then multiply by rebounds by 1.5, if contains F, then points by 1.5 (it may contain more than one letter ex: G-F )
player_teams['ppg'] = round(player_teams['points']/player_teams['GP'],2)
player_teams['rpg'] = round(player_teams['rebounds']/player_teams['GP'],2)
player_teams['apg'] = round(player_teams['assists']/player_teams['GP'],2)
player_teams['spg'] = round(player_teams['steals']/player_teams['GP'],2)
player_teams['bpg'] = round(player_teams['blocks']/player_teams['GP'],2)

#efficiency
player_teams['eff'] = player_teams['ppg'] + player_teams['rpg'] + player_teams['apg'] + player_teams['spg'] + player_teams['bpg'] - (player_teams['fgAttempted'] - player_teams['fgMade']) - (player_teams['ftAttempted'] - player_teams['ftMade']) - player_teams['turnovers']

#per 36 minutes stats
player_teams['pp36'] = compute_percentage(player_teams['points'], player_teams['minutes'])*36

#defensive prowess: Defensive Prowess PCA: Use 'steals', 'blocks', and 'dRebounds' to create a 'Defensive Impact' principal component. Combine 'PF' (personal fouls) and 'turnovers' into a 'Defensive Discipline' component to represent careful play.
player_teams['defensive_prowess'] = compute_percentage(player_teams['steals'] + player_teams['blocks'] + player_teams['dRebounds'], player_teams['GP'])*10
player_teams['defensive_discipline'] = compute_percentage(player_teams['PF'] + player_teams['turnovers'], player_teams['GP'])*2

#minutes per game
player_teams['mpg'] = compute_percentage(player_teams['minutes'], player_teams['GP'])

#percetange of game started
player_teams['gs%'] = compute_percentage(player_teams['GS'], player_teams['GP'])
    

#add pos column from players to player_teams, bioID is the same as playerID
player_teams = player_teams.merge(players[['bioID', 'pos', 'college']], left_on='playerID', right_on='bioID', how='left')
player_teams.drop('bioID', axis=1, inplace=True)

player_teams['pos'] = player_teams['pos'].replace(
    ['G', 'F', 'C', 'C-F', 'F-C', 'G-F', 'F-G'],
    [1, 2, 3, 4, 4, 5, 5]
)

#turn this oRebounds,dRebounds,dq,PostMinutes,PostPoints,PostoRebounds,PostdRebounds,PostRebounds,PostAssists,PostSteals,PostBlocks,PostTurnovers,PostPF,PostDQ into per game stats
player_teams['oRebounds'] = round(player_teams['oRebounds']/player_teams['GP'],2)
player_teams['dRebounds'] = round(player_teams['dRebounds']/player_teams['GP'],2)
player_teams['dq'] = round(player_teams['dq']/player_teams['GP'],2)
player_teams['PostMinutes'] = round(player_teams['PostMinutes']/player_teams['PostGP'],2)
player_teams['PostPoints'] = round(player_teams['PostPoints']/player_teams['PostGP'],2)
player_teams['PostoRebounds'] = round(player_teams['PostoRebounds']/player_teams['PostGP'],2)
player_teams['PostdRebounds'] = round(player_teams['PostdRebounds']/player_teams['PostGP'],2)
player_teams['PostRebounds'] = round(player_teams['PostRebounds']/player_teams['PostGP'],2)
player_teams['PostAssists'] = round(player_teams['PostAssists']/player_teams['PostGP'],2)
player_teams['PostSteals'] = round(player_teams['PostSteals']/player_teams['PostGP'],2)
player_teams['PostBlocks'] = round(player_teams['PostBlocks']/player_teams['PostGP'],2)
player_teams['PostTurnovers'] = round(player_teams['PostTurnovers']/player_teams['PostGP'],2)
player_teams['PostPF'] = round(player_teams['PostPF']/player_teams['PostGP'],2)
player_teams['PostDQ'] = round(player_teams['PostDQ']/player_teams['PostGP'],2)

#remove the made and attempted columns
player_teams.drop([ 'threeMade', 'threeAttempted', 'PostftMade', 'PostftAttempted', 'PostfgMade', 'PostfgAttempted', 'PostthreeMade', 'PostthreeAttempted', 'PostGS', 'PostGP'], axis=1, inplace=True)

#remove the columns that are not needed: points, rebounds, assists, steals, blocks, turnovers, PF, fgMade, fgAttempted, ftMade, ftAttempted, GS, GP, minutes
player_teams.drop([ 'points', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgMade', 'fgAttempted', 'ftMade', 'ftAttempted', 'GS', 'minutes'], axis=1, inplace=True)

#fill nan with 0
player_teams.fillna(0, inplace=True)

player_teams.head()
player_teams.columns

Index(['playerID', 'year', 'stint', 'tmID', 'GP', 'oRebounds', 'dRebounds',
       'dq', 'PostMinutes', 'PostPoints', 'PostoRebounds', 'PostdRebounds',
       'PostRebounds', 'PostAssists', 'PostSteals', 'PostBlocks',
       'PostTurnovers', 'PostPF', 'PostDQ', 'ft%', 'fg%', 'three%', 'gs%',
       'Postft%', 'Postfg%', 'Postthree%', 'Postgs%', 'efg%', 'ts%', 'ppg',
       'rpg', 'apg', 'spg', 'bpg', 'eff', 'pp36', 'defensive_prowess',
       'defensive_discipline', 'mpg', 'pos', 'college'],
      dtype='object')

In [40]:
coaches['total_games'] = coaches['won']+ coaches['lost']
coaches['W%'] = compute_percentage(coaches['won'],coaches['total_games'])

coaches['total_p_games'] = coaches['post_wins']+ coaches['post_losses']
coaches['postW%'] = compute_percentage(coaches['post_wins'],coaches['total_p_games'])

#remove won and lost columns, and post_wins and post_losses
coaches.drop(['won','lost','post_wins','post_losses'], axis=1, inplace=True)

coaches.head()

In [ ]:
def pre_process_data(df):
    mapping = {'Y': 1, 'N': 0}
    df['playoff'] = df['playoff'].map(mapping)
    df.fillna(0, inplace=True)
    return df

In [ ]:
teams = pre_process_data(teams)

In [ ]:
#just get the columns tmID, year, playoff, confID, firstRound, semis, finals
teams_playoffs = teams[['tmID', 'year', 'playoff', 'confID', 'firstRound', 'semis', 'finals']]

#merge teams_playoffs with player_teams, based on tmID and year, and add the playoff columns to player_teams
player_teams = player_teams.merge(teams_playoffs, left_on=['tmID', 'year'], right_on=['tmID', 'year'], how='left')

#add champions column to player_teams
player_teams['champions'] = player_teams.apply(
    lambda row: 'W' if row['finals'] == 'W' else '',
    axis=1
)
# Create a new column for playoff_progression
player_teams['playoff_progression'] = player_teams.apply(
    lambda row: '1' if row['firstRound'] == 'L' else
                '2' if row['semis'] == 'L' else
                '3' if row['finals'] == 'L' else
                '4' if row['champions'] == 'W' else '0',
    axis=1
)

# Drop unnecessary columns
player_teams.drop(['firstRound', 'semis', 'finals', 'champions'], axis=1, inplace=True)

player_teams.head()

player_teams['confID'].unique()

array(['WE', 'EA'], dtype=object)

In [ ]:
#add height and weight columns to player_teams from players
player_teams = player_teams.merge(players[['bioID', 'height', 'weight']], left_on='playerID', right_on='bioID', how='left')

#remove bioID column
player_teams.drop('bioID', axis=1, inplace=True)

player_teams.head()

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,defensive_prowess,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression,height,weight
0,abrossv01w,2,0,MIN,26,1.65,5.04,0.08,0.0,0.00,...,70.0,11.92,32.54,2,Connecticut,0,WE,0,74.0,169
1,abrossv01w,3,0,MIN,27,1.67,3.74,0.00,0.0,0.00,...,56.7,12.22,29.81,2,Connecticut,0,WE,0,74.0,169
2,abrossv01w,4,0,MIN,30,1.47,3.23,0.00,23.0,7.67,...,50.7,11.26,26.40,2,Connecticut,1,WE,1,74.0,169
3,abrossv01w,5,0,MIN,22,0.77,2.59,0.00,33.5,10.00,...,40.5,7.72,21.00,2,Connecticut,1,WE,1,74.0,169
4,abrossv01w,6,0,MIN,31,0.94,2.52,0.00,0.0,0.00,...,42.6,10.70,25.06,2,Connecticut,0,WE,0,74.0,169


In [ ]:
#from awards_players, add column award_count to player_teams which is the number of awards the team won (sum of all awards)
#player_teams = player_teams.merge(awards_players[['playerID', 'award']], left_on='playerID', right_on='playerID', how='left')

player_teams['award_count'] = 0

#associate each player with each award and year (ex: player A won 2 awards in 2010, thus 2010 has 2, but 2011 is 0 (unless he wins again))
for index, row in awards_players.iterrows():
    player_teams.loc[(player_teams['playerID'] == row['playerID']) & (player_teams['year'] == row['year']), 'award_count'] += 1

player_teams.head()

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,defensive_discipline,mpg,pos,college,playoff,confID,playoff_progression,height,weight,award_count
0,abrossv01w,2,0,MIN,26,1.65,5.04,0.08,0.0,0.00,...,11.92,32.54,2,Connecticut,0,WE,0,74.0,169,0
1,abrossv01w,3,0,MIN,27,1.67,3.74,0.00,0.0,0.00,...,12.22,29.81,2,Connecticut,0,WE,0,74.0,169,0
2,abrossv01w,4,0,MIN,30,1.47,3.23,0.00,23.0,7.67,...,11.26,26.40,2,Connecticut,1,WE,1,74.0,169,0
3,abrossv01w,5,0,MIN,22,0.77,2.59,0.00,33.5,10.00,...,7.72,21.00,2,Connecticut,1,WE,1,74.0,169,0
4,abrossv01w,6,0,MIN,31,0.94,2.52,0.00,0.0,0.00,...,10.70,25.06,2,Connecticut,0,WE,0,74.0,169,0


In [ ]:
#encode playerID, teamID, college
player_teams = encode_player_team(player_teams)
player_teams.to_csv('new_data/clean-data.csv', index=False)

In [ ]:
#add ['height', 'weight', 'career_year', 'pos', 'college','confID'] to comp.csv

comp = comp.merge(players[['bioID', 'height', 'weight', 'pos', 'college']], left_on='playerID', right_on='bioID', how='left')
comp.drop(['bioID','lgID'], axis=1, inplace=True)


#make pos into a category
comp['pos'] = comp['pos'].replace(
    ['G', 'F', 'C', 'C-F', 'F-C', 'G-F', 'F-G'],
    [1, 2, 3, 4, 4, 5, 5]
)

#get confID from teams
team_conf = teams[['tmID', 'confID']]
#get unique pairs of tmID and confID
team_conf = team_conf.drop_duplicates()
comp = comp.merge(team_conf, left_on='tmID', right_on='tmID', how='left')

#if confID is nan, then print the tmID
for index, row in comp.iterrows():
    if row['confID'] != row['confID']:
        #set confID to WE
        comp.loc[index, 'confID'] = 'WE'

In [ ]:
#store comp dataframe into a csv file
comp = encode_player_team(comp)
comp.to_csv('new_data/clean-comp.csv', index=False)

In [ ]:
#join comp to player_teams, if a column does not exist, fill it with 0, player_teams is from year 1-10, comp is from year 11
new_player_teams = player_teams.merge(comp, left_on=['playerID','year','stint','tmID','height','weight','pos','college','confID'], right_on=['playerID','year','stint','tmID','height','weight','pos','college','confID'], how='outer')
new_player_teams.fillna(0, inplace=True)
#sort by year
new_player_teams = new_player_teams.sort_values(by=['year','playerID'])

#drop any _y columns or _x columns
new_player_teams = new_player_teams.drop([col for col in new_player_teams.columns if col.endswith('_y')], axis=1)
new_player_teams = new_player_teams.drop([col for col in new_player_teams.columns if col.endswith('_x')], axis=1)

new_player_teams['career_year'] = new_player_teams.groupby('playerID').cumcount() + 1

new_player_teams

,playerID,year,stint,tmID,GP,oRebounds,dRebounds,dq,PostMinutes,PostPoints,...,mpg,pos,college,playoff,confID,playoff_progression,height,weight,award_count,career_year
12,4,1,0,559,29.0,0.28,0.55,0.00,24.50,7.00,...,9.38,1,579,1.0,702,1,71.0,153,0.0,1
28,12,1,0,559,30.0,0.40,1.13,0.00,15.00,1.00,...,14.47,1,586,1.0,702,1,67.0,125,0.0,1
31,13,1,0,566,32.0,1.00,1.97,0.03,27.67,3.50,...,24.91,2,587,1.0,702,2,71.0,165,0.0,1
37,15,1,0,558,32.0,1.12,2.59,0.03,33.50,8.33,...,30.53,1,589,1.0,701,4,71.0,147,0.0,1
42,16,1,0,567,32.0,3.00,4.22,0.00,0.00,0.00,...,27.16,3,589,0.0,702,0,77.0,198,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,741,11,0,559,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,1,598,0.0,702,0,69.0,145,0.0,1
2005,742,11,0,573,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,3,598,0.0,701,0,77.0,195,0.0,1
2006,742,11,0,563,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,3,598,0.0,701,0,77.0,195,0.0,2
2010,743,11,0,574,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,1,750,0.0,702,0,68.0,130,0.0,1


In [ ]:
cols = ['playoff_progression', 'playoff', 'pp36','eff', 'award_count','defensive_prowess', 'defensive_discipline']
new_cols = [f'{col}_rolling' for col in cols]

new_player_teams = new_player_teams.groupby('playerID').apply(rolling_averages, cols, new_cols, window=2)

In [ ]:
#store new_player_teams into a csv file
new_player_teams.to_csv('new_data/complete-data.csv', index=False)